# Crypto Lab 2: Putting it all together

**Prerequisites:**
- Complete all parts 1-5 of this lab
- Complemte the IoT lab

Now the time has come to put everything together. Your task is to create your own secret messaging scheme for the Micro:Bit.

- Part 1: Verify that the radio works. I.e., two microbits can communicate.
- Part 2: Using ChaCha20, we begin securing the communication between two Micro:Bits. We use a static key to begin with and provide code for the receiver-side. 
- Part 3: Extend with MAC functionality. We provide a custom hashing algortihm for the MAC.  
- Part 4: Extend so that the key is no longer static, but computed using DHKE. 



---

> **NB: Make sure you have completed Crypto Lab 1: Basic introductions before starting on this lab**

In this lab, you’ll take what you learned in the previous crypto lab and bring it to life on real hardware using the **Micro:Bit**. You’ll also be introduced to a new algorithm: **ChaCha20**. 

To follow along, you’ll need two Micro:Bits - ask your TA's if you only have one. 

## 1 Verify that two Micro:Bits can communicate using the Radio

First, let’s make sure the two Micro:Bits can communicate with each other!  
Copy the code provided in [lab2-1.py](/crypto-lab/lab-2/lab2-1.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3) and upload it to **both** of your devices.  

Next, choose one Micro:Bit to act as the **sender** and the other as the **receiver**.  
- On the sender, press the buttons to transmit a message.  
- On the receiver, check that the message is displayed correctly.  

> ⚠️ **Important:** Before you run the program, update the `GROUP_NUMBER` in the code to match your assigned rat group.  
> Your group number is created by combining your team number with A/B (where A = 0 and B = 1).  

Examples:  
- Team **1A** → `10`  
- Team **1B** → `11` 

#### Question 1.1: Explain briefly what happens in the code snippet (maximum 3 lines of text)

#### Question 1.2: Right now, the messages are sent “in the clear” over the radio - what does this mean?

#### Question 1.3: What happens if another team nearby sets the same group number?

Before moving on, make sure the setup works:  
Press **A** or **B** on the sender, and check that the corresponding message shows up on the receiver’s display.  


---

## 2 Encryption and decryption of messages

Now that you’ve confirmed the two Micro:Bits can communicate, it’s time to take the next step: implementing a simple **encryption and decryption** mechanism.  

1. Copy the code from [lab2-2.py](/crypto-lab/lab-2/lab2-2.py) into the [Micro:Bit Web Editor](https://python.microbit.org/v/3).  
2. Send the code to both Micro:Bits.  
3. ⚠️ Don’t forget to update the `GROUP_NUMBER` here as well, just like in Task 1.  

If you look at the new code, you’ll notice that most of the previous functions remain unchanged.  
However, a few **new elements have been added**!

If you scroll down to the bottom of the Python file, you’ll find a **ChaCha20 module**.  
Don’t worry if it looks complex, you are **not expected to fully understand it** in this course - however it will become relevant in later cryptographic courses.

As seen in the preparation material, ChaCha20 is a widely used stream cipher.

#### Question 2.1: Why does ChaCha20 only have an encrypt function, not decrypt? 

You’ll also notice some **new constants** in the code:  

- `GLOBAL_KEY` → a static global key written in binary format  
- `MESSAGE_1` / `MESSAGE_2` → example messages you can use to test the functionality  

💡 **Why this matters:** In real-world cryptography, securely sharing and managing keys is one of the hardest challenges.  
For now, we use a fixed key for simplicity, but later you’ll see how key exchange and management become critical for secure communication.  

In [9]:
GLOBAL_KEY = b'chacha20!'
MESSAGE_1 = "Hello World"
MESSAGE_2 = "Goodbye World"

More importantly, both functions:`def send_mode()` and `def receive_mode()` are altered. They both call a new function:`on_receive()` before displaying the data: 

In [ ]:
def on_receive(received_bytes):
    try: 
        decrypted_bytes = chacha20_encrypt(
            received_bytes, GLOBAL_KEY)
        data = decrypted_bytes.decode('utf-8')
        return data
    except TypeError:
        return "TYPE ERROR"
    except:
        return "WEIRD ERROR"

#### Question 2.2: Explain shortly what happens in the `on_receive` function.

Additionally, `def send_mode()` has been altered to send the messages written as constants instead of "A" or "B" as in part 1. Here, a new function `on_send()` is used instead of `radio.send()`.

Using the `on_receive()` function, let's try to fill out `on_send()`!   
Follow the steps as written in the TODO:

In [ ]:
def on_send(msg):
    """
    TODO: 
    Implement a send function that encodes the inputted
    message to bytes, encrypts the bytes using chacha20_encrypt()
    and then sends the encrypted bytes over the radio.
    """
    pass # Remove this line and implement your solution here

<details>
<summary><strong>💡 Hint</strong></summary>

Try using the same logic as in on_receive() to fill out on_send(). NOEN BEDRE HINT HER???
</details>

If your code works as intended, you should now be able to send encrypted messages. Verify that it works 

## 3 Extend with MAC technology

